In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import binom

In [52]:
def call(strike):
    return lambda x: max(0, x - strike)

def put(strike):
    return lambda x: max(0, strike - x)

def long(strike):
    return lambda x: x - strike

def short(strike):
    return lambda x: strike - x

def straddle(strike):
    return lambda x: call(strike)(x) + put(strike)(x)

In [48]:
class QuizPlayer():
    def __init__(self, total_questions):
        self.total_questions = total_questions
        
        self.answered = 0
        self.correct = 0
        self.score = 0
        
        self.ledger = []
    
    def __repr__(self):
        return (f'has answered {self.answered} questions out of {self.total_questions} and '
                f'got {self.correct} correct. Score is {self.score}')
    
    
    def _round_update(self, correct, questions):
        self.correct += correct
        self.answered += questions
        self.score += correct
    
    
    def round_update(self, correct, questions):
        self._round_update(correct, questions)
        self.ledger.append((correct, questions))
        print(f'got {correct}/{questions} this round')
    
        
    def undo(self):
        try:
            args = self.ledger.pop(-1)
            self._round_update(-args[0], -args[1])
            print(f'Undo: round_update({args[0]}, {args[1]})')
        except:
            print('nothing to undo')

    def price(self, option):
        # note: binom.pmf(x, n, p, starting point)
        remaining = self.total_questions - self.answered
        try:
            accuracy = self.correct / self.answered
        except:
            accuracy = 0.5
        return sum([option(x) * binom.pmf(x, remaining, accuracy, self.score)
                   for x in range(-self.total_questions, 2*self.total_questions)]).round()
    
    def shock(self, score):
        self.score += score

In [49]:
awais = QuizPlayer(20)

In [50]:
awais.round_update(5, 6)

got 5/6 this round


In [53]:
awais.price(straddle(10))

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 6.380467472191411e-11, 3.573061784427214e-09, 8.709338099541337e-08, 1.1612450799388402e-06, 7.983559924579507e-06, 0.0, 0.0005987669943434649, 0.006843051363925309, 0.044907524575759916, 0.19958899811448852, 0.6237156191077761, 1.3608340780533323, 1.9845496971611156, 1.744659074427353, 0.7009790924038447, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


7.0

In [45]:
awais

has answered 16 questions out of 20 and got 11 correct. Score is 11